# **Using HST COS data for Lyα** (Sujay)
### We only use FUVM coadded spectra

In [ ]:
import altair as alt
import corv
import numpy as np
import pandas as pd

from astropy.table import Table
from astroquery.gaia import Gaia as GQ
from astroquery.simbad import Simbad as SQ
from pathlib import Path
from scripts import make_bins
from tqdm import tqdm
from typing import Tuple

In [128]:
cos_objs = pd.read_csv('data/hst_cos/STAR_WD_COS_new.info', sep=r'\s,\s+', engine='python', usecols=[1, 2, 3], names=['ID', 'FLAG', 'META'])
cos_objs = cos_objs[cos_objs.META.str.contains('DA')]
cos_objs = cos_objs[~cos_objs.META.str.contains('PULSAR')].ID

gaia_ids = {}
for obj in cos_objs:
    query = SQ.query_objectids(obj).to_pandas().ID.str.decode('utf-8')
    gaia_ids[int(query[query.str.contains('DR3')].iloc[0].replace('Gaia DR3 ', ''))] = obj

params = GQ.launch_job(f'SELECT source_id, radial_velocity, teff_gspphot, logg_gspphot FROM gaiadr3.gaia_source WHERE source_id in {tuple(gaia_ids.keys())}').get_results().to_pandas().dropna()
params.SOURCE_ID.replace(gaia_ids, inplace=True)
params.to_csv('data/hst_cos/cos_params.csv', index=False)

In [139]:
params.query('SOURCE_ID == "TYC 7218-934-1"').teff_gspphot.iloc[0]

5518.9727

In [121]:
cos_objs[137]
SQ.query_objectids('WD1547+057').to_pandas().ID.str.decode('utf-8')


0       LAMOST J154934.76+053518.2
1         SDSS J154934.75+053517.1
2           GALEX J154934.7+053518
3     Gaia DR3 4426435871273922560
4                    TIC 272675206
5                      PG 1547+057
6                      WD 1547+057
7         SDSS J154934.77+053517.1
8          USNO-B1.0 0955-00254275
9          2MASS J15493479+0535169
10        SDSS J154934.78+053517.1
11    Gaia DR2 4426435871273922560
Name: ID, dtype: object

In [146]:
tbl = Table.read('data/hst_cos/TYC-7218-934-1_coadd_FUVM_final_lpALL.fits.gz').to_pandas()
tbl

,WAVE,FLUX,ERROR,FLUXERR_UP,FLUXERR_DOWN,GROSSCOUNTS,NETCOUNTS,NETCOUNTSERR_UP,NETCOUNTSERR_DOWN,BACK,EXP_PIX,DQ,SN,FLUXFACTOR,FLUXWGT,FLUXWGT_ERR,CHI2,EXP_COUNTER
0,892.479760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,892.489720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,892.499679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,892.509639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,892.519598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58953,1479.627443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
58954,1479.637403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
58955,1479.647362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
58956,1479.657322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
